In [1]:
import pandas as pd
from coypu.utils.query import Query
from coypu.utils.credentials import *
from query.queries import *
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"
import matplotlib.pyplot as plt
import numpy as np
from coypu.utils.functions import json_to_csv, timer
from json import loads, load
import os
from functools import reduce

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

coypu.utils


In [ ]:
cmemc_query_imp = Query(client_url_imp, client_id_imp, client_secret_imp , 'oauth')
    
fuseki_infai = Query(fuseki_endpoint_infai, fuseki_user_infai, fuseki_pw_infai, 'basic')

fdquery = Query("http://localhost:5002/sparql", is_fdq=True)

In [ ]:
print(query_4_fdq_desc)
print(query_4_fdq)
print('########################### EXAMPLE ##########################\n{}'.format(query_4_fdq_ex))

# fdquery.post_query(query_4_fdq_ex)
# fdquery.to_save('../../query_results/', filename='query_4_fdq_ex')

In [ ]:

@timer
def load_json(filename):
    with open(filename) as js:
        json_data = load(js)
        return json_data

@timer
def merge_jsons_to_csv(filepath, save_path, outfile, columns):
    df = reduce(lambda df1, df2: pd.concat([df1, df2]), \
                [json_to_csv(load_json(filepath+filename)['results']['bindings'], save_path, filename, columns, True) for filename in os.listdir(filepath)])
    
    df.to_csv(save_path+outfile, encoding='utf-8', index=False)


In [ ]:

    
columns=[var+'.value' for var in load_json('../../query_results/query_4/query4_germany.isocode.json')['head']['vars']]
print(columns)
# merge_jsons_to_csv('../../query_results/query_4/', '../../query_results/', 'query_4_fdq_ex.csv', columns)

json_to_csv(load_json('../../query_results/query_4/query4_syria.isocode.json')['results']['bindings'], '../../query_results/', 'query_4_fdq_syria_ex', columns)

In [ ]:
def timestamp_to_year(text):
    return pd.Timestamp(text).year

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(
        "mysql+pymysql://{user}:{pw}@{hostname}:{portno}/{db}".format(
            hostname=host, db=dbname, user=uname, pw=pwd, portno=port
        ),)

filepath = '../../query_results/'
for filename in os.listdir(filepath):
    if 'query4_' in filename:
        print(filename)
        df = pd.read_csv(filepath+filename, encoding='utf-8')
        df.rename(columns=lambda x: x.replace('.value', ''), inplace=True)
        df['year']= df['time'].apply(timestamp_to_year)
        try:
            df.to_sql('query_4_fdq_ex', con=engine, if_exists='append', index=False, chunksize=300000)
        except Exception as e:
            print('Exeception: {}',e)

In [2]:
from sqlalchemy import create_engine

engine = create_engine(
        "mysql+pymysql://{user}:{pw}@{hostname}:{portno}/{db}".format(
            hostname=host, db=dbname, user=uname, pw=pwd, portno=port))
df = pd.read_sql(query_4_fdq_ex_sql, con=engine)

df.to_csv('../../query_results/query_4_fdq_ex_sql.csv', encoding='utf-8', index=False)

In [6]:
df = pd.read_csv('../../query_results/query_4_fdq_ex_sql.csv', encoding='utf-8')

df

,isocode,country,year,fatalities_per_million,no_of_events
0,UKR,Ukraine,2021,30.0678,30620
1,UKR,Ukraine,2020,0.0000,2
2,UKR,Ukraine,2019,0.0000,11
